# Part 1

In [6]:
import os
import re
from collections import Counter
import numpy as np
from matplotlib import pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
from plotly import plot
from tensorflow.python.ops.numpy_ops import equal

In [2]:
def load_file(file_path: str) -> np.ndarray:
    """
    Loads the input file and parses each line to extract complex numbers p and v.

    Parameters:
        file_path (str): Path to the input data file.

    Returns:
        np.ndarray: 2D NumPy array with shape (N, 2) where each row contains p and v as complex numbers.
    """
    data_pairs = []

    # Define the regex pattern
    # Adjust the pattern if your numbers can be floating-point
    pattern = r'p=(?P<p_real>-?\d+),(?P<p_imag>-?\d+)\s+v=(?P<v_real>-?\d+),(?P<v_imag>-?\d+)'
    regex = re.compile(pattern)

    with open(file_path, 'r') as file:
        for line_number, line in enumerate(file, start=1):
            line = line.strip()
            if not line:
                continue  # Skip empty lines
            match = regex.match(line)
            if match:
                try:
                    # Extract and convert to floats
                    p_real = float(match.group('p_real'))
                    p_imag = float(match.group('p_imag'))
                    v_real = float(match.group('v_real'))
                    v_imag = float(match.group('v_imag'))

                    # Create complex numbers
                    p = complex(p_real, p_imag)
                    v = complex(v_real, v_imag)

                    # Append as a row
                    data_pairs.append([p, v])
                except ValueError as ve:
                    print(f"Value conversion error on line {line_number}: {ve}")
            else:
                print(f"Line {line_number} doesn't match the expected format: {line}")

    if not data_pairs:
        print("No valid data pairs found.")
        return np.array([])  # Return empty array if no data

    # Convert list of lists to 2D NumPy array
    data_array = np.array(data_pairs, dtype=complex)
    return data_array

In [4]:
def wrap_around(coord, min_val, max_val):
    """
    Wraps coordinates that exceed the specified bounds using modulo operation.

    Parameters:
        coord (np.ndarray): Array of coordinate values (real or imaginary parts).
        min_val (float): Minimum allowed value.
        max_val (float): Maximum allowed value.

    Returns:
        np.ndarray: Wrapped coordinate values within bounds.
    """
    range_val = max_val - min_val
    wrapped = (coord - min_val) % range_val + min_val
    return wrapped

In [17]:
def plot_complex_with_counts_plotly_express(data_array: np.ndarray, dim_start=0+0j, dim_end=7+11j):
    """
    Plots complex numbers with counts using Plotly Express for interactivity.

    Parameters:
        data_array (np.ndarray): 2D NumPy array with shape (N, 2) where each row contains p and v as complex numbers.
        dim_start (complex): Starting point defining plot's lower bounds (default: 0+0j).
        dim_end (complex): Ending point defining plot's upper bounds (default: 7+11j).
    """
    # Ensure that data_array is a 2D array with at least one column
    if data_array.ndim != 2 or data_array.shape[1] < 1:
        raise ValueError("data_array must be a 2D NumPy array with at least one column for 'p' values.")

    # Extract p values (assuming 'p' is in the first column)
    p_values = data_array[:, 0]

    # Convert complex numbers to tuples (real, imag) with rounding to handle floating-point precision
    p_tuples = [ (round(p.real, 5), round(p.imag, 5)) for p in p_values ]

    # Count occurrences of each unique p value
    overlap_counter = Counter(p_tuples)

    # If no p values to plot, exit the function
    if not overlap_counter:
        print("No p values to plot.")
        return

    # Extract unique points and their counts
    points = np.array(list(overlap_counter.keys()))
    counts = np.array(list(overlap_counter.values()))

    # Create a DataFrame for Plotly Express
    df = pd.DataFrame({
        'Real': points[:, 0],
        'Imaginary': points[:, 1],
        'Count': counts
    })

    # Create scatter plot with Plotly Express
    fig = px.scatter(
        df,
        x='Real',
        y='Imaginary',
        size='Count',
        color='Count',
        hover_data=['Count'],
        title='Complex Numbers p with Overlapping Counts',
        range_x=[dim_start.real - 1, dim_end.real + 1],
        range_y=[dim_end.imag + 1, dim_start.imag - 1],
        color_continuous_scale='Viridis',
        size_max=20,

    )

    fig.update_layout(
        yaxis=dict(
            scaleanchor="x",
            scaleratio=1,
            title='Imaginary Part'
        ),
        xaxis=dict(
            title='Real Part'
        ),
        legend_title_text='Count',
    )



    # Show the plot inline
    fig.show()

In [55]:
input_file = 'in.txt'  # Replace with your input file path

# print current working directory
print("Current working directory: ", os.getcwd())


data_array = load_file(input_file)

if data_array.size == 0:
    print("No data to process. Exiting.")

# Display the structured array
print("Structured 2D NumPy Array (p and v as complex numbers):")
print(data_array)

# test dimension size
dim_start, dim_end = 0 + 0j, 101 + 103j
second_count = 100

plot_complex_with_counts_plotly_express(data_array, dim_start, dim_end)
#1.12.......
#...........
#...........
#......11.11
#1.1........
#.........1.
#.......1...

Current working directory:  /home/rob/Programming/aoc2024/AdventOfCode2024/days/day14
Structured 2D NumPy Array (p and v as complex numbers):
[[ 40. +73.j -96. +64.j]
 [ 74. +78.j  91. -65.j]
 [100. +86.j  98. +62.j]
 [ 61. +29.j  95. -68.j]
 [ 50. +53.j -50. -63.j]
 [ 43. +77.j -42. -37.j]
 [ 47. +44.j  34. +66.j]
 [ 11. +21.j  72. +66.j]
 [  9. +61.j -26. +77.j]
 [ 46.+100.j -10. -82.j]
 [ 64. +15.j  -5. -44.j]
 [ 50. +97.j -56. +37.j]
 [ 37. +35.j -95. +80.j]
 [ 33.  +8.j  60. +39.j]
 [ 56. +81.j  38.  +7.j]
 [ 60. +22.j -11. +69.j]
 [ 47. +36.j -55. +33.j]
 [ 29. +51.j -39. -69.j]
 [ 93. +89.j -96. -14.j]
 [ 67. +51.j -80. -78.j]
 [ 68.+102.j -46. +46.j]
 [ 66. +51.j  -9. +40.j]
 [ 95. +28.j  73.  -1.j]
 [ 41. +86.j -97. +81.j]
 [ 15. +54.j  16. -33.j]
 [ 68. +47.j  -7. -20.j]
 [ 75. +34.j  40. +23.j]
 [  4. +38.j  27. +73.j]
 [ 14. +78.j -82. -49.j]
 [ 23. +68.j  63. +87.j]
 [ 82. +69.j  83. +24.j]
 [  4. +38.j  62. +14.j]
 [ 60. +65.j  46. +71.j]
 [ 12. +14.j  13. -31.j]
 [ 14. +

In [56]:
for _ in range(second_count):

    # we need to check if a new position is within the dimension,
    # if not then the new position is going to on the other side of the dimension
    # moved by the remaining steps specified by the velocity


    # move all the robots
    data_array[:, 0] += data_array[:, 1]

    # wrap around the new positions
    data_array[:, 0] = wrap_around(data_array[:, 0].real, dim_start.real, dim_end.real) + wrap_around(data_array[:, 0].imag, dim_start.imag, dim_end.imag) * 1j

plot_complex_with_counts_plotly_express(data_array, dim_start, dim_end)
data_array

#..... 2..1.
#..... .....
#1.... .....
#
#..... .....
#...12 .....
#.1... 1....

array([[ 35. +87.j, -96. +64.j],
       [ 84. +67.j,  91. -65.j],
       [  2.  +3.j,  98. +62.j],
       [ 67. +27.j,  95. -68.j],
       [100. +36.j, -50. -63.j],
       [ 85. +85.j, -42. -37.j],
       [ 13. +52.j,  34. +66.j],
       [ 40. +29.j,  72. +66.j],
       [ 35. +36.j, -26. +77.j],
       [ 56. +37.j, -10. -82.j],
       [ 69. +44.j,  -5. -44.j],
       [  5. +89.j, -56. +37.j],
       [ 31.  +1.j, -95. +80.j],
       [ 74. +97.j,  60. +39.j],
       [ 18. +60.j,  38.  +7.j],
       [ 71. +21.j, -11. +69.j],
       [  1. +40.j, -55. +33.j],
       [ 68. +52.j, -39. -69.j],
       [ 88. +28.j, -96. -14.j],
       [ 46. +79.j, -80. -78.j],
       [ 13. +67.j, -46. +46.j],
       [ 75. +34.j,  -9. +40.j],
       [ 22. +31.j,  73.  -1.j],
       [ 37. +49.j, -97. +81.j],
       [100. +50.j,  16. -33.j],
       [ 75.  +4.j,  -7. -20.j],
       [ 35. +68.j,  40. +23.j],
       [ 78. +25.j,  27. +73.j],
       [ 96. +19.j, -82. -49.j],
       [ 61. +13.j,  63. +87.j],
       [10

In [57]:
# we need to split the data_array into 4 new 2D arrays, 1 for each quadrant. If the dimensions are odd number, we throw away the middle row and column

q1 = []
q2 = []
q3 = []
q4 = []

for i in range(data_array.shape[0]):

    if data_array[i, 0].real < dim_start.real + (dim_end.real - dim_start.real) // 2:
        if data_array[i, 0].imag > dim_start.imag + (dim_end.imag - dim_start.imag) // 2:
            # create a new 2D array for quadrant 1
            q1.append(data_array[i])
        elif data_array[i, 0].imag < dim_start.imag + (dim_end.imag - dim_start.imag) // 2:
            q3.append(data_array[i])
    elif data_array[i, 0].real > dim_start.real + (dim_end.real - dim_start.real) // 2:
        if data_array[i, 0].imag > dim_start.imag + (dim_end.imag - dim_start.imag) // 2:
            q2.append(data_array[i])
        elif data_array[i, 0].imag < dim_start.imag + (dim_end.imag - dim_start.imag) // 2:
            q4.append(data_array[i])


q1 = np.array(q1)
q2 = np.array(q2)
q3 = np.array(q3)
q4 = np.array(q4)

q1, q2, q3, q4

(array([[ 35. +87.j, -96. +64.j],
        [ 13. +52.j,  34. +66.j],
        [  5. +89.j, -56. +37.j],
        [ 18. +60.j,  38.  +7.j],
        [ 46. +79.j, -80. -78.j],
        [ 13. +67.j, -46. +46.j],
        [ 35. +68.j,  40. +23.j],
        [ 43. +99.j,  62. +14.j],
        [ 14. +58.j,  46. +71.j],
        [ 31. +85.j,  37. +91.j],
        [ 28.+102.j, -92. -68.j],
        [ 21. +56.j,  77. -70.j],
        [ 33. +93.j, -27.  -2.j],
        [ 13. +55.j, -19.  -7.j],
        [ 43. +56.j, -63. -99.j],
        [ 32. +97.j,  12. -39.j],
        [ 48. +70.j, -99.  -3.j],
        [  1. +86.j, -50. +42.j],
        [ 20. +62.j,  46. +41.j],
        [ 17. +66.j, -58. +30.j],
        [ 48. +62.j,   1. -84.j],
        [ 24. +56.j,  75. -55.j],
        [  8. +61.j, -19. +85.j],
        [ 13. +99.j,  46. -13.j],
        [  0. +98.j, -51. +72.j],
        [ 32. +76.j, -89. +86.j],
        [ 24.+102.j,  83. +44.j],
        [ 48. +96.j, -99. -98.j],
        [ 23. +60.j,   7. -45.j],
        [ 39. 

In [58]:
# plot the 4 quadrants
plot_complex_with_counts_plotly_express(q1, dim_start, dim_end)
plot_complex_with_counts_plotly_express(q2, dim_start, dim_end)
plot_complex_with_counts_plotly_express(q3, dim_start, dim_end)
plot_complex_with_counts_plotly_express(q4, dim_start, dim_end)

In [59]:
# multiply the counts of each item in each quadrant and rule out the one with the lowest product
# lets get the length of each of the quadrants
q1_len = q1.shape[0]
q2_len = q2.shape[0]
q3_len = q3.shape[0]
q4_len = q4.shape[0]

q1_len, q2_len, q3_len, q4_len




(110, 105, 140, 135)

In [60]:
# multiply the counts of each item in each quadrant
product = q1_len * q2_len * q3_len * q4_len
product

218295000

# PART 2

In [64]:
import math
from functools import reduce

In [79]:
def generate_christmas_tree(dim_start, dim_end):
    """
    Generates target positions forming a Christmas tree within the defined grid.

    Parameters:
        dim_start (complex): Starting point defining plot's lower bounds (e.g., 0+0j).
        dim_end (complex): Ending point defining plot's upper bounds (e.g.,7+11j).

    Returns:
        list of complex: List of complex positions representing the Christmas tree.
    """
    x_min = int(dim_start.real)
    x_max = int(dim_end.real)
    y_min = int(dim_start.imag)
    y_max = int(dim_end.imag)

    tree_positions = []

    trunk_height = 2
    body_height = y_max - trunk_height  # y=0 to y_max - trunk_height -1

    for y in range(y_min, y_max + 1):
        if y < y_max - trunk_height:
            # Tree body
            if y <= 6:
                width = y + 1  # y=0:1, y=1:2, ..., y=6:7
            else:
                width = 7  # y=7,8,9:7
        else:
            # Trunk
            width = 2

        # Calculate x positions centered at 3.5
        x_center = 3.5
        half_width = width / 2

        # Determine starting x-coordinate
        x_start = math.floor(x_center - half_width + 0.5)
        x_end = x_start + width

        # Ensure x positions are within grid boundaries
        x_start = max(x_min, x_start)
        x_end = min(x_max + 1, x_end)  # range is up to x_end -1

        x_positions = list(range(x_start, x_end))

        # Add positions to the list
        for x in x_positions:
            pos = complex(x, y)
            tree_positions.append(pos)

    return tree_positions


In [86]:
import random


def generate_christmas_tree(dim_start, dim_end):
    """
    Generates target positions forming a Christmas tree within the defined grid.

    Parameters:
        dim_start (complex): Starting point defining plot's lower bounds (e.g., 0+0j).
        dim_end (complex): Ending point defining plot's upper bounds (e.g.,7+11j).

    Returns:
        list of complex: List of complex positions representing the Christmas tree's outer layers.
    """
    x_min = int(math.floor(dim_start.real))
    x_max = int(math.ceil(dim_end.real))
    y_min = int(math.floor(dim_start.imag))
    y_max = int(math.ceil(dim_end.imag))

    tree_positions = []

    trunk_height = 3
    body_height = y_max - trunk_height  # y=0 to y_max - trunk_height -1

    for y in range(y_min, y_max + 1):
        if y < y_max - trunk_height:
            # Tree body
            if y <= 4:
                width = y + 1  # y=0:1, y=1:2, ..., y=4:5
            else:
                width = 7  # y=5,6,7,8:7
        else:
            # Trunk
            width = 2

        # Calculate x positions centered at 3.5
        x_center = (x_min + x_max) / 2
        half_width = width / 2

        # Determine starting x-coordinate
        x_start = math.floor(x_center - half_width + 0.5)
        x_end = x_start + width

        # Ensure x positions are within grid boundaries
        x_start = max(x_min, x_start)
        x_end = min(x_max + 1, x_end)  # range is up to x_end -1

        x_positions = list(range(x_start, x_end))

        # Identify outer layer positions
        if y < y_max - trunk_height:
            # For body, include only the perimeter points
            if width == 1:
                # Top point
                tree_positions.append(complex(x_positions[0], y))
            else:
                # Add left and right points of the row
                tree_positions.append(complex(x_positions[0], y))
                tree_positions.append(complex(x_positions[-1], y))
        else:
            # For trunk, include all points (since it's narrow)
            for x in x_positions:
                tree_positions.append(complex(x, y))

    return tree_positions

def select_70_percent_positions(positions):
    """
    Selects approximately 70% of the provided positions.

    Parameters:
        positions (list of complex): List of complex positions.

    Returns:
        list of complex: Subset of positions selected to be occupied.
    """
    total_positions = len(positions)
    num_selected = math.ceil(0.7 * total_positions)  # Round up to ensure at least 70%

    selected_positions = random.sample(positions, num_selected)
    return selected_positions

def plot_tree_plotly(selected_positions, dim_start, dim_end):
    """
    Plots the Christmas tree using Plotly Express for an interactive visualization.

    Parameters:
        selected_positions (list of complex): List of complex positions selected to be occupied.
        dim_start (complex): Starting point defining plot's lower bounds.
        dim_end (complex): Ending point defining plot's upper bounds.
    """
    # Create a DataFrame
    df_tree = pd.DataFrame({
        'Real': [pos.real for pos in selected_positions],
        'Imaginary': [pos.imag for pos in selected_positions]
    })

    # Create scatter plot with Plotly Express
    fig = px.scatter(
        df_tree,
        x='Real',
        y='Imaginary',
        title='Christmas Tree Pattern (70% Occupied Outer Layers)',
        range_x=[dim_start.real - 1, dim_end.real + 1],
        range_y=[dim_start.imag - 1, dim_end.imag + 1],
        color_discrete_sequence=['green'],
        size_max=12,
        hover_data=['Real', 'Imaginary']
    )

    # Update layout for equal aspect ratio
    fig.update_layout(
        yaxis=dict(
            scaleanchor="x",
            scaleratio=1,
            title='Imaginary Part'
        ),
        xaxis=dict(
            title='Real Part'
        ),
        showlegend=False,
        width=600,
        height=800
    )

    # Display the plot
    fig.show()



In [87]:
def find_min_time(initial_p, target_p, velocity, dim_start, dim_end):
    """
    Finds the minimal time for a robot to reach its target position.

    Parameters:
        initial_p (complex): Initial position of the robot.
        target_p (complex): Target position of the robot.
        velocity (complex): Velocity of the robot.
        dim_start (complex): Starting point defining plot's lower bounds.
        dim_end (complex): Ending point defining plot's upper bounds.

    Returns:
        int or None: Minimal time in seconds to reach target, or None if impossible.
    """
    if velocity.real == 0 and velocity.imag == 0:
        # Robot doesn't move
        if initial_p.real == target_p.real and initial_p.imag == target_p.imag:
            return 0
        else:
            return None  # Impossible to reach

    width = dim_end.real - dim_start.real
    height = dim_end.imag - dim_start.imag

    times_real = []
    times_imag = []

    # Calculate possible times for x-axis
    if velocity.real != 0:
        delta_x = (target_p.real - initial_p.real) % width
        t_x = delta_x / velocity.real
        if t_x < 0:
            t_x += math.ceil(abs(t_x))
        t_x_ceil = math.ceil(t_x)
        times_real.append(t_x_ceil)
    else:
        # Velocity in x is 0; check if already at target
        if (initial_p.real % width) != (target_p.real % width):
            return None  # Impossible to reach
        else:
            times_real.append(0)

    # Calculate possible times for y-axis
    if velocity.imag != 0:
        delta_y = (target_p.imag - initial_p.imag) % height
        t_y = delta_y / velocity.imag
        if t_y < 0:
            t_y += math.ceil(abs(t_y))
        t_y_ceil = math.ceil(t_y)
        times_imag.append(t_y_ceil)
    else:
        # Velocity in y is 0; check if already at target
        if (initial_p.imag % height) != (target_p.imag % height):
            return None  # Impossible to reach
        else:
            times_imag.append(0)

    # Find common times where both axes are satisfied
    possible_times = set(times_real) & set(times_imag)
    if possible_times:
        return min(possible_times)
    else:
        return None


In [91]:
def calculate_overall_min_time(robots, dim_start, dim_end, target_positions, required_percentage=0.7):
    """
    Calculates the overall minimal synchronization time for robots to reach their targets.

    Parameters:
        robots (list of tuples): Each tuple contains (initial_p, velocity) as complex numbers.
        dim_start (complex): Starting point defining plot's lower bounds.
        dim_end (complex): Ending point defining plot's upper bounds.
        target_positions (list of complex): Target positions for the robots.
        required_percentage (float): Percentage of robots that need to reach their targets.

    Returns:
        int or None: Minimal synchronization time in seconds, or None if impossible.
    """
    individual_times = []
    for (initial_p, velocity), target_p in zip(robots, target_positions):
        t = find_min_time(initial_p, target_p, velocity, dim_start, dim_end)
        if t is None:
            print(f"Robot starting at {initial_p} with velocity {velocity} cannot reach target {target_p}.")
            continue  # Exclude robots that cannot reach their target
        individual_times.append(t)

    if not individual_times:
        return None  # No robots can reach their targets

    # Determine the number of robots required to reach their targets
    total_reachable = len(individual_times)
    required_reachable = math.ceil(required_percentage * total_reachable)

    # Sort the times in ascending order
    sorted_times = sorted(individual_times)

    # The minimal synchronization time is the time at the required_reachable-th position
    T_min = sorted_times[required_reachable - 1]

    return T_min


In [92]:
dim_start = 0 + 0j
dim_end = 7 + 11j

input_file = 'test.txt'
robots = load_file(input_file)


# Set random seed for reproducibility
random.seed(42)

# Define grid dimensions
dim_start = 0 + 0j
dim_end = 7 + 11j

# Generate all outer positions of the Christmas tree
all_outer_positions = generate_christmas_tree(dim_start, dim_end)

# Select approximately 70% of the outer positions
selected_positions = select_70_percent_positions(all_outer_positions)

# Display the selected positions
print(f"Total Outer Positions: {len(all_outer_positions)}")
print(f"Selected Positions (70%): {len(selected_positions)}")
print("Selected Positions:")
for pos in sorted(selected_positions, key=lambda p: (p.imag, p.real)):
    print(f"({int(pos.real)}, {int(pos.imag)})")

# Assign robots to selected target positions
if len(robots) != len(selected_positions):
    print(f"Number of robots ({len(robots)}) does not match number of target positions ({len(selected_positions)}).")

# Calculate minimal synchronization time
T_min = calculate_overall_min_time(robots, dim_start, dim_end, selected_positions, required_percentage=0.7)
if T_min is not None:
    print(f"\nThe minimal synchronization time is {T_min} seconds.")
else:
    print("\nSynchronization into a Christmas tree pattern is impossible with the given robot configurations.")

# Plot the Christmas tree
plot_tree_plotly(selected_positions, dim_start, dim_end)






Total Outer Positions: 23
Selected Positions (70%): 17
Selected Positions:
(3, 0)
(3, 1)
(2, 2)
(4, 2)
(5, 3)
(1, 4)
(5, 4)
(0, 5)
(6, 5)
(0, 6)
(6, 6)
(3, 9)
(4, 9)
(3, 10)
(4, 10)
(3, 11)
(4, 11)
Number of robots (12) does not match number of target positions (17).
Robot starting at 4j with velocity (3-3j) cannot reach target (4+10j).
Robot starting at (6+3j) with velocity (-1-3j) cannot reach target (2+2j).
Robot starting at (10+3j) with velocity (-1+2j) cannot reach target (3+0j).
Robot starting at (2+0j) with velocity (2-1j) cannot reach target (5+4j).
Robot starting at 0j with velocity (1+3j) cannot reach target (1+4j).
Robot starting at (7+6j) with velocity (-1-3j) cannot reach target (4+2j).
Robot starting at (9+3j) with velocity (2+3j) cannot reach target (6+5j).
Robot starting at (7+3j) with velocity (-1+2j) cannot reach target 6j.
Robot starting at (2+4j) with velocity (2-3j) cannot reach target (3+10j).

The minimal synchronization time is 1 seconds.


In [94]:

dim_start, dim_end = 0 + 0j, 101 + 103j

input_file = 'in.txt'
robots = load_file(input_file)


# Set random seed for reproducibility
random.seed(42)


# Generate all outer positions of the Christmas tree
all_outer_positions = generate_christmas_tree(dim_start, dim_end)

# Select approximately 70% of the outer positions
selected_positions = select_70_percent_positions(all_outer_positions)

# Display the selected positions
print(f"Total Outer Positions: {len(all_outer_positions)}")
print(f"Selected Positions (70%): {len(selected_positions)}")
print("Selected Positions:")
for pos in sorted(selected_positions, key=lambda p: (p.imag, p.real)):
    print(f"({int(pos.real)}, {int(pos.imag)})")

# Assign robots to selected target positions
if len(robots) != len(selected_positions):
    print(f"Number of robots ({len(robots)}) does not match number of target positions ({len(selected_positions)}).")

# Calculate minimal synchronization time
T_min = calculate_overall_min_time(robots, dim_start, dim_end, selected_positions, required_percentage=0.7)
if T_min is not None:
    print(f"\nThe minimal synchronization time is {T_min} seconds.")
else:
    print("\nSynchronization into a Christmas tree pattern is impossible with the given robot configurations.")

# Plot the Christmas tree
plot_tree_plotly(selected_positions, dim_start, dim_end)

Total Outer Positions: 207
Selected Positions (70%): 145
Selected Positions:
(50, 1)
(51, 2)
(52, 3)
(48, 4)
(52, 4)
(47, 6)
(53, 7)
(47, 9)
(53, 9)
(47, 10)
(53, 10)
(53, 11)
(47, 12)
(53, 12)
(47, 13)
(53, 13)
(47, 14)
(53, 14)
(47, 15)
(47, 16)
(53, 16)
(47, 17)
(53, 17)
(47, 18)
(47, 20)
(53, 20)
(47, 21)
(47, 22)
(53, 23)
(53, 24)
(47, 25)
(53, 25)
(47, 26)
(47, 27)
(53, 27)
(47, 28)
(53, 28)
(47, 29)
(53, 29)
(47, 30)
(53, 31)
(47, 32)
(47, 33)
(47, 34)
(53, 34)
(53, 35)
(47, 36)
(53, 36)
(53, 37)
(47, 38)
(47, 41)
(53, 41)
(47, 42)
(47, 43)
(53, 43)
(47, 44)
(53, 44)
(53, 45)
(47, 46)
(53, 46)
(47, 47)
(53, 47)
(53, 48)
(47, 49)
(53, 49)
(47, 50)
(47, 51)
(47, 52)
(53, 52)
(47, 53)
(53, 53)
(47, 54)
(53, 54)
(53, 55)
(53, 57)
(47, 58)
(53, 58)
(47, 59)
(53, 59)
(47, 60)
(53, 61)
(47, 62)
(53, 62)
(47, 63)
(53, 63)
(47, 65)
(53, 65)
(53, 67)
(47, 69)
(53, 69)
(47, 70)
(53, 70)
(47, 71)
(53, 71)
(47, 72)
(53, 72)
(47, 73)
(53, 73)
(47, 74)
(53, 74)
(47, 75)
(53, 75)
(47, 76)
(47, 